Listy chromosome i mRNA utworzone! Pierwszym krokiem będzie projektowanie starterów dla chromosome, a mRNA wstrzymam się do opracowania 

In [1]:
import pandas as pd
import numpy as np
import csv
import matplotlib as mpl 
import matplotlib.pyplot as plt

### deseq OUT

In [4]:
# podejcie MW #
#--------------#
# wyniki deseq:
# pvalue jest normalizowane do padj przez "the Holm method" http://www.math.ucla.edu/~anderson/rw1001/library/base/html/p.adjust.html
# w analizie odciecie jest przy padj = 0.001 (mozliwe bylo w DEBROWSER) i taki mam plik wynikowy, podzielony na UP/DOWN
# (w zasadzie dwa pliki), z ktorych na podstawie najwyzszego log2foldchange wzialem po 100 najwyzszch sekwencji,
# polaczylem razem i zblastowalem. Plik ktory jest ponizej to zlepek 100UP i 100DOWN wynikow deseq, tych samych, ktore sie blastowaly

In [2]:
df_deseq_out = pd.read_csv('/mnt/sdb1/Project_Sieja_MCLR_52/2_pipeline/7_counting_transcripts/noncoding_novel/opracowane_deseq/podejcie_MW/6d/PBS1d_vs_MC6d_DESEQ_100UP_100DOWN_together.csv')
df_deseq_out.head()

,ID,PBS_1d_1,PBS_1d_2,PBS_1d_3,PBS_1d_4,MC_6d_1,MC_6d_2,MC_6d_3,padj,log2FoldChange,pvalue,foldChange,log10padj
0,TRINITY_DN89469_c3_g1_i4,3508.639738,286.889935,424.439079,2739.108727,0.0,0.000000,0.0,3.427124e-14,"-13,3824580501411",1.308537e-15,0.000094,13.465070
1,TRINITY_DN108781_c6_g7_i1,741.399314,351.089081,1921.491644,1564.921640,0.0,0.000000,0.0,1.157249e-22,"-12,7785433017495",1.695150e-24,0.000142,21.936573
2,TRINITY_DN82922_c1_g2_i3,898.567298,1025.180116,790.750842,807.253622,0.0,0.000000,0.0,2.377948e-74,"-12,3999122849851",4.496204e-78,0.000185,73.623798
3,TRINITY_DN83880_c4_g1_i13,303.496795,866.688474,1992.779861,227.102063,0.0,0.000000,0.0,2.906981e-15,"-12,3448124542481",9.681932e-17,0.000192,14.536558
4,TRINITY_DN105991_c2_g2_i7,2727.135490,4919.259580,2264.771829,1599.631563,0.0,1.835489,0.0,3.636601e-38,"-12,2457290826912",1.161373e-40,0.000206,37.439304


# filtrowanie pliku wynikowego blasta

In [3]:
df_blast_results = pd.read_csv('/mnt/sdb1/Project_Sieja_MCLR_52/2_pipeline/7_counting_transcripts/noncoding_novel/opracowane_deseq/podejcie_MW/6d/blast_all_200.csv', sep='\t', low_memory=False)
df_blast_results.shape

(1040247, 5)

### odfiltrowanie zbyt niskiego coverage

In [5]:
df_blast_results.columns

Index(['query', 'al_title', 'al_accession', 'evalue', 'coverage'], dtype='object')

In [4]:
df_blast_results_20cov = df_blast_results[df_blast_results['coverage'] > 20]
df_blast_results_20cov.head()

,query,al_title,al_accession,evalue,coverage
0,TRINITY_DN80414_c8_g1_i6,gi|1711367508|emb|LR664381.1| Coregonus sp. 'b...,LR664381,0.000000e+00,100.571429
1,TRINITY_DN80414_c8_g1_i6,gi|1614056723|emb|LR584425.1| Salmo trutta gen...,LR584425,0.000000e+00,106.171429
2,TRINITY_DN80414_c8_g1_i6,gi|1614056709|emb|LR584411.1| Salmo trutta gen...,LR584411,0.000000e+00,67.428571
5,TRINITY_DN80414_c8_g1_i6,gi|1348554173|ref|XM_024138321.1| PREDICTED: S...,XM_024138321,6.688140e-53,30.285714
9,TRINITY_DN92637_c2_g3_i1,gi|1711366356|emb|LR664357.1| Coregonus sp. 'b...,LR664357,0.000000e+00,99.838710


In [7]:
df_blast_results_20cov.shape

(423547, 5)

### filtrowanie mRNA only

In [7]:
# select what you need based on string
df_mRNA_only = df_blast_results_20cov[df_blast_results_20cov['al_title'].str.contains('mRNA')]
df_mRNA_only.shape


(1575, 5)

In [9]:
df_mRNA_only.to_csv('/mnt/sdb1/Project_Sieja_MCLR_52/2_pipeline/7_counting_transcripts/noncoding_novel/opracowane_deseq/podejcie_MW/6d/PBS1d_vs_MCLR6d_mRNA_only.csv', index=False)

In [8]:
# select best hit based on evalue (drop_duplicates)
df_mRNA_only_best_hit = df_mRNA_only.sort_values('evalue', ascending=True).drop_duplicates('query').sort_index()


In [9]:
df_mRNA_only_best_hit.shape

(130, 5)

In [10]:
df_mRNA_only_best_hit.sort_values(by='coverage', ascending=False, inplace=True)

In [13]:
# save df as file
df_mRNA_only_best_hit.to_csv('/mnt/sdb1/Project_Sieja_MCLR_52/2_pipeline/7_counting_transcripts/noncoding_novel/opracowane_deseq/podejcie_MW/6d/PBS1d_vs_MCLR6d_mRNA_only_best_hit.csv', index=False)

In [14]:
# łączenie odfiltrowanych mRNA z wynikami deseq

df_mRNA_only_best_hit.rename(columns={'query' : 'ID'}, inplace=True)
df_mRNA_only_best_hit.head()

,ID,al_title,al_accession,evalue,coverage
264807,TRINITY_DN98234_c1_g1_i1,gi|1367536286|ref|XM_024397182.1| PREDICTED: O...,XM_024397182,4.885760e-94,110.017889
791353,TRINITY_DN81156_c15_g1_i1,gi|929276519|ref|XM_014159553.1| PREDICTED: Sa...,XM_014159553,2.422560e-47,108.830549
63868,TRINITY_DN89088_c2_g2_i1,gi|1696032593|ref|XM_029743272.1| PREDICTED: S...,XM_029743272,2.778670e-131,103.648069
175321,TRINITY_DN106993_c0_g3_i1,gi|1696170136|ref|XM_029715903.1| PREDICTED: S...,XM_029715903,7.671280e-137,103.519669
46044,TRINITY_DN100272_c6_g1_i1,gi|1158853613|ref|XM_020490808.1| PREDICTED: O...,XM_020490808,0.000000e+00,101.626016


In [15]:
result = pd.merge(df_deseq_out, df_mRNA_only_best_hit, on='ID')
result.sort_values(by='log2FoldChange', inplace=True)

In [16]:
result.to_csv('/mnt/sdb1/Project_Sieja_MCLR_52/2_pipeline/7_counting_transcripts/noncoding_novel/opracowane_deseq/podejcie_MW/6d/PBS_1d_vs_MC2_6d_deseq_concat_with_mRNA_best_hit.csv', index=False)

### filtrowanie chromosomes

In [ ]:
# zgodnie z zasada [wszystkie - mRNA = reszta, czyli chromosomes]

In [11]:
# best hit z calych wynikow blasta

df_best_hit_all = df_blast_results_20cov.sort_values('evalue', ascending=True).drop_duplicates('query').sort_index()
df_best_hit_all.shape

(199, 5)

In [12]:
# df1[~df1.isin(df2)].dropna()
# jakos na zasadzie, jesli cos jest w jednym to usun to z drugiego
df_non_mRNA_best_hit = df_best_hit_all[~df_best_hit_all['query'].isin(df_mRNA_only_best_hit['query'])]

# sortowanie 
df_non_mRNA_best_hit.sort_values(by='coverage', ascending=False, inplace=True)

# zapis
df_non_mRNA_best_hit.to_csv('/mnt/sdb1/Project_Sieja_MCLR_52/2_pipeline/7_counting_transcripts/noncoding_novel/opracowane_deseq/podejcie_MW/6d/PBS1d_vs_MCLR6d_chromosome_only_best_hit.csv', index=False)

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [13]:
df_non_mRNA_best_hit.rename(columns={'query' : 'ID'}, inplace=True)
df_non_mRNA_best_hit.head()

/usr/lib/python3.7/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,ID,al_title,al_accession,evalue,coverage
793485,TRINITY_DN99723_c8_g1_i2,gi|1711367680|emb|LR664382.1| Coregonus sp. 'b...,LR664382,2.838270e-116,104.074074
136062,TRINITY_DN101788_c3_g1_i1,gi|1614056736|emb|LR584438.1| Salmo trutta gen...,LR584438,0.000000e+00,103.287982
569886,TRINITY_DN113838_c9_g1_i3,gi|1711366400|emb|LR664358.1| Coregonus sp. 'b...,LR664358,2.209700e-112,102.492212
212253,TRINITY_DN93401_c2_g8_i1,gi|1614056714|emb|LR584416.1| Salmo trutta gen...,LR584416,0.000000e+00,102.445652
972419,TRINITY_DN99250_c2_g1_i11,gi|1711366093|emb|LR664351.1| Coregonus sp. 'b...,LR664351,9.784010e-146,102.318841


In [14]:
### łączenie odfiltrowanych chromosomes z wynikami deseq

result2 = pd.merge(df_deseq_out, df_non_mRNA_best_hit, on='ID')
result2.sort_values(by='log2FoldChange', inplace=True)

In [15]:
result2.to_csv('/mnt/sdb1/Project_Sieja_MCLR_52/2_pipeline/7_counting_transcripts/noncoding_novel/opracowane_deseq/podejcie_MW/6d/PBS_1d_vs_MC2_6d_deseq_concat_with_chromosome_best_hit.csv', index=False)
